In [2]:
# Import Required Libraries
import requests
import plotly.express as px
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error

In [3]:
# Authenticate and Retrieve Portfolio

import robin_stocks.robinhood as r

# Define a function to authenticate with Robinhood API
def authenticate_robinhood(username, password):
    login = r.login(username, password)
    return login

# Define a function to retrieve the current portfolio tickers
def get_portfolio_tickers():
    positions_data = r.account.build_holdings()
    tickers = list(positions_data.keys())
    return tickers

['META', 'AAPL', 'SPOT', 'NVDA', 'SHOP', 'GOOGL', 'BRK.B', 'RBLX', 'GS', 'Z', 'LLY', 'AVAV', 'ARM', 'SMR', 'VRSK', 'JPM', 'TGS', 'MU', 'CEG']


In [30]:
from datetime import timezone


def linear_func(x, m, c):
    return m * x + c

def exponential_func(x, a, b):
    return a * np.exp(b * x)


#Define a function to calculate metrics for each ticker for a given time range
def calculate_regression_metrics_for_range(historical_data, start_date):
    results = []
    for ticker, data in historical_data.items():
        
        # Convert start_date to the same timezone as data.index
        start_date = pd.to_datetime(start_date).tz_convert(data.index.tz)
        
        # Filter data for the given time range
        data_range = data[data.index >= start_date]
        
        if data_range.empty:
            continue
        
        # Convert date to ordinal for fitting
        data_range['date'] = data_range.index.to_series().apply(lambda date: date.toordinal())
        x_data = data_range['date']
        y_data = data_range['close_price'].astype(float)

        # Fit the linear curve
        popt_linear = np.polyfit(x_data, y_data, 1)
        
        # Fit the exponential curve
        popt_exp, _ = curve_fit(exponential_func, x_data, y_data, maxfev=10000)
        
        # Calculate metrics for linear fit
        y_pred_linear = linear_func(x_data, *popt_linear)
        slope = popt_linear[0]
        variance = np.var(y_pred_linear)
        std_dev = np.std(y_pred_linear)
        max_drawdown = np.min(y_pred_linear) - np.max(y_pred_linear)
        
        # Calculate metrics for exponential fit
        exp_param = popt_exp[1]  # Exponential growth rate
        
        # Append results
        results.append({
            'ticker': ticker,
            'slope': slope,
            'exp_param': exp_param,
            'variance': variance,
            'standard_deviation': std_dev,
            'maximum_drawdown': max_drawdown
        })
    

    
    return pd.DataFrame(results)

# Define a function to calculate metrics for multiple time ranges
def calculate_metrics_for_multiple_ranges(historical_data):
    time_ranges = {
        '5_years': datetime.now(tz=timezone.utc) - timedelta(days=5*365),
        '3_years': datetime.now(tz=timezone.utc) - timedelta(days=3*365),
        '1_year': datetime.now(tz=timezone.utc) - timedelta(days=365),
        '6_months': datetime.now(tz=timezone.utc) - timedelta(days=6*30),
        '3_months': datetime.now(tz=timezone.utc) - timedelta(days=3*30)
    }
    aggregated_results = pd.DataFrame()
    
    for range_name, start_date in time_ranges.items():
        metrics_df = calculate_regression_metrics_for_range(historical_data, start_date)
        metrics_df['time_range'] = range_name
        aggregated_results = pd.concat([aggregated_results, metrics_df], ignore_index=True)
    
    return aggregated_results



In [80]:
import os
import time
import pickle
from datetime import datetime, timedelta

# Fetch Historical Data

# Define a function to fetch historical data for a given ticker
def fetch_historical_data(ticker, interval='day', span='5year'):
    time.sleep(1)
    historical_data = r.stocks.get_stock_historicals(ticker, interval=interval, span=span)
    historical_data = pd.DataFrame(historical_data)
    historical_data.index = pd.to_datetime(historical_data['begins_at']).dt.tz_convert('UTC')
    return historical_data

# Define a function to load data from cache if it exists and is younger than 1 day
def load_from_cache(ticker):
    print("Loading from cache")
    cache_file = f"{ticker}_historical_data.pkl"
    if os.path.exists(cache_file):
        file_time = datetime.fromtimestamp(os.path.getmtime(cache_file))
        if datetime.now() - file_time < timedelta(days=1):
            with open(cache_file, 'rb') as f:
                return pickle.load(f)
    return None

# Define a function to save data to cache
def save_to_cache(ticker, data):
    print("Saving to cache")
    cache_file = f"{ticker}_historical_data.pkl"
    with open(cache_file, 'wb') as f:
        pickle.dump(data, f)


In [83]:
def calulate_time_slope_delta_plot(grouped_data, group_title):

    # Calculate metrics for multiple time ranges
    aggregated_results = calculate_metrics_for_multiple_ranges(grouped_data)

    pivoted_results = aggregated_results.pivot(index='ticker', columns='time_range', values=['slope', 'exp_param', 'variance', 'standard_deviation', 'maximum_drawdown'])
    pivoted_results

    aggregated_results.sort_values('slope', ascending=False)

    # Sort the time ranges
    time_ranges_order = ['3_months', '6_months', '1_year', '3_years', '5_years']
    aggregated_results['time_range'] = pd.Categorical(aggregated_results['time_range'], categories=time_ranges_order, ordered=True)
    sorted_results = aggregated_results.sort_values('time_range', ascending=False)

    # Create the plot
    fig = px.line(sorted_results, x='time_range', y='slope', color='ticker', markers=True)

    fig.update_layout(
        title=f"Slope of Different Tickers Over Various Time Ranges {group_title}",
        xaxis_title="Time Range",
        yaxis_title="Slope"
    )

    return fig, sorted_results


In [ ]:
# Example usage
username = input("Enter your Robinhood username: ")
password = input("Enter your Robinhood password: ")
authenticate_robinhood(username, password)

In [ ]:
def run_stats_for_group(group_name):

    if group_name == 'portfolio':
        tickers = get_portfolio_tickers()
        print(tickers)

        historical_data = {}
        for ticker in tickers:
            data = load_from_cache(ticker)
            if data is None:
                data = fetch_historical_data(ticker)
                save_to_cache(ticker, data)
            historical_data[ticker] = data

    else:
        tickers = stock_lists[group_name]
        print(tickers)

        historical_data = {}
        for ticker in tickers:
            data = load_from_cache(ticker)
            if data is None:
                data = fetch_historical_data(ticker)
                save_to_cache(ticker, data)
            historical_data[ticker] = data
    
    return historical_data

In [67]:
historical_data = {}

historical_data['portfolio'] = run_stats_for_group("portfolio")
calulate_time_slope_delta_plot(historical_data, group_title="Portfolio")

['META', 'AAPL', 'SPOT', 'NVDA', 'SHOP', 'GOOGL', 'BRK.B', 'RBLX', 'GS', 'Z', 'LLY', 'AVAV', 'ARM', 'SMR', 'VRSK', 'JPM', 'TGS', 'MU', 'CEG']
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache


/tmp/ipykernel_1221821/1362272647.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda/envs/azureml_py38/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning:

overflow encountered in exp

/anaconda/envs/azureml_py38/lib/python3.9/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:

Covariance of the parameters could not be estimated



In [76]:
# Define a function to get all stock lists from Robinhood
def get_all_stock_lists():
    watchlists = r.account.get_all_watchlists()['results']
    stock_lists = {}
    for watchlist in watchlists:
        try:
            list_name = watchlist['display_name']
            print(list_name)
            results = r.account.get_watchlist_by_name(list_name)['results']
            symbols = [item['symbol'] for item in results]
            stock_lists[list_name] = symbols
        except Exception as e:
            print(f"Error fetching watchlist {list_name}: {e}")
    return stock_lists

# Example usage
stock_lists = get_all_stock_lists()




Post Stonks
Dividend
Biotech
Energy
Options Watchlist
400 Client Error: Bad Request for url: https://api.robinhood.com/midlands/lists/items/?list_id=633d6309-6671-4fb2-95bb-35357476d1d2
Error fetching watchlist Options Watchlist: 'NoneType' object is not subscriptable
Bonds
Banks, Finance and Homes
High debt load
Massachusetts
Daily Movers
QT Resilient
GPT
Long term bets
Covid Boom Bust
2024 Q1 short
2024 long term
2024 2dm
2024, Fall, Short Term, Long
Nuclear


In [78]:
stock_lists

{'Post Stonks': ['SMR',
  'IOT',
  'SHOP',
  'ASML',
  'ADSK',
  'RDW',
  'RKLB',
  'NOW',
  'MGA',
  'COST',
  'APTV',
  'GM',
  'DDOG',
  'IBB',
  'ENTG',
  'IVV',
  'CCI',
  'GOLF',
  'IRT'],
 'Dividend': ['JEPI',
  'XYLD',
  'RYLD',
  'SHW',
  'DGRO',
  'DLR',
  'IRM',
  'MRK',
  'VZ',
  'CTAS',
  'WSBF',
  'NEM',
  'MPLX',
  'AGNC',
  'O',
  'RWT'],
 'Biotech': ['JNJ',
  'AMGN',
  'GILD',
  'AZN',
  'PFE',
  'MRNA',
  'CYRX',
  'VRTX',
  'BMY'],
 'Energy': ['CEG', 'SMR', 'OKLO', 'COP', 'XLE'],
 'Bonds': ['IBHE', 'TIP'],
 'Banks, Finance and Homes': ['AAL',
  'GS',
  'C',
  'KBH',
  'SEDG',
  'CNXC',
  'JBGS',
  'SNV',
  'OZK',
  'WAL',
  'FITB',
  'FHN',
  'EWBC',
  'XRT',
  'USB',
  'ABCB',
  'CATY',
  'FFIN',
  'CBSH',
  'WBS',
  'FIBK',
  'LNC',
  'CRBG',
  'BAC',
  'EMB',
  'HYG',
  'SPG',
  'CACC',
  'ALLY',
  'COF',
  'RF',
  'FRCB',
  'CMBS',
  'TSLA',
  'VNO',
  'BXP',
  'NYMT',
  'DX',
  'SLG',
  'KRE'],
 'High debt load': ['VSCO', 'KMX', 'VZ', 'T', 'CMCSA', 'NFLX', 'CHTR

In [74]:
stock_lists.keys()

AttributeError: 'str' object has no attribute 'keys'

In [84]:
figs = []
slopes = []

for stock_list in stock_lists.keys():
    historical_data[stock_list] = run_stats_for_group(stock_list)
    fig, slope_df = calulate_time_slope_delta_plot(historical_data[stock_list], group_title=stock_list)
    figs.append(fig)
    slopes.append(slope_df)


#historical_data[group_name] = run_stats_for_group(group_name)
#calulate_time_slope_delta_plot(historical_data, group_title=group_name)

['SMR', 'IOT', 'SHOP', 'ASML', 'ADSK', 'RDW', 'RKLB', 'NOW', 'MGA', 'COST', 'APTV', 'GM', 'DDOG', 'IBB', 'ENTG', 'IVV', 'CCI', 'GOLF', 'IRT']
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache


/tmp/ipykernel_1221821/1362272647.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/anaconda/envs/azureml_py38/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning:

overflow encountered in exp

/anaconda/envs/azureml_py38/lib/python3.9/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:

Covariance of the parameters could not be estimated



['JEPI', 'XYLD', 'RYLD', 'SHW', 'DGRO', 'DLR', 'IRM', 'MRK', 'VZ', 'CTAS', 'WSBF', 'NEM', 'MPLX', 'AGNC', 'O', 'RWT']
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
['JNJ', 'AMGN', 'GILD', 'AZN', 'PFE', 'MRNA', 'CYRX', 'VRTX', 'BMY']
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
['CEG', 'SMR', 'OKLO', 'COP', 'XLE']
Loading from cache
Loading from cache
Loading from cache
Loading from cache
Loading from cache
['IBHE', 'TIP']
Loading from cache
Loading from cache
['AAL', 'GS', 'C', 'KBH', 'SEDG', 'CNXC', 'JBGS', 'SNV', 'OZK', 'WAL', 'FITB', 'FHN', 'EWBC', 'XRT', 'USB', 'ABCB', 'CATY', 'FFIN', 'CBSH', 'WBS', 'FIBK

In [82]:
for fig in figs:
    fig.show()

In [95]:
slopes_df = pd.concat(slopes)

In [96]:
slopes_df = slopes_df.drop_duplicates(subset=['ticker', 'time_range'])

In [98]:
slopes_df

,ticker,slope,exp_param,variance,standard_deviation,maximum_drawdown,time_range
0,SMR,0.005007,1.0,6.916598,2.629943,-9.113214,5_years
10,APTV,-0.024528,1.0,165.961378,12.882600,-44.640474,5_years
1,IOT,0.010246,1.0,28.957911,5.381255,-18.646996,5_years
18,IRT,0.002640,1.0,1.923116,1.386765,-4.805383,5_years
17,GOLF,0.019586,1.0,105.826595,10.287205,-35.646976,5_years
...,...,...,...,...,...,...,...
11,UEC,0.001353,1.0,0.019710,0.140392,-0.487183,1_year
16,UEC,0.020333,1.0,1.042809,1.021180,-3.558287,6_months
17,CCJ,0.105643,1.0,28.150044,5.305662,-18.487499,6_months
21,UEC,-0.007201,1.0,0.032656,0.180708,-0.612062,3_months


In [97]:
len(slopes_df)

1855